In [13]:
import json
import os
import csv

# Define the paths
jsonl_file = "/Users/jianfengzhu/Desktop/LLMs-in-IR-SEP7/Human_xBenchmarks_164/humaneval.jsonl"
output_dir = "/Users/jianfengzhu/Desktop/LLMs-in-IR-SEP7/processed_data/Human_x_164_O3"

# Process the input JSONL file
with open(jsonl_file, 'r') as file:
    for line in file:
        # Parse each line as a JSON object
        data = json.loads(line)
        
        # Extract task_id and format it
        task_id = data["task_id"].replace("/", "_")
        task_file_name = task_id + ".csv"
        
        # Check if the corresponding file exists in the output directory
        task_dir = os.path.join(output_dir, task_id)
        if os.path.exists(task_dir):
            # Extract assert statements from the test section
            test_code = data.get("test", "")
            assert_statements = [line.strip() for line in test_code.splitlines() if line.strip().startswith("assert")]
            
            # Define the CSV file path
            output_csv_file = os.path.join(task_dir, task_file_name)
            
            # Write the assert statements to the CSV file
            with open(output_csv_file, 'w', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerow(["task_id", "assert_statement"])
                for assert_statement in assert_statements:
                    csv_writer.writerow([task_id, assert_statement])

print("Extraction and saving of assert statements are complete.")


Extraction and saving of assert statements are complete.


In [1]:
import os
import random
import openai
import pandas as pd
import os

# 设置环境变量
TOGETHER_API_KEY="6c82971bb71dcd6e4862ad4584bddac579f03cefe027709048843146abced619"

# 初始化 Groq 客户端
import os
from together import Together

client = Together(api_key=TOGETHER_API_KEY)


In [ ]:
import os
import shutil

# Define the source and destination directories
source_dir = "/Users/jianfengzhu/Desktop/LLMs-in-IR-SEP7/Human_xBenchmarks_164"
destination_dir = "/Users/jianfengzhu/Desktop/LLMs-in-IR-SEP7/processed_data/Human_x_164_O3"

# Get a list of all subfolders in the source directory
subfolders = [f for f in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, f))]

# Iterate through each subfolder
for subfolder in subfolders:
    # Define the full path to the .cpp file in the source directory
    cpp_file = os.path.join(source_dir, subfolder, f"{subfolder}.cpp")
    
    # Define the destination subfolder path
    dest_subfolder = os.path.join(destination_dir, subfolder)
    
    # Check if the .cpp file exists in the source directory
    if os.path.exists(cpp_file):
        # Copy the .cpp file to the corresponding destination subfolder
        shutil.copy(cpp_file, dest_subfolder)
        print(f"Copied {cpp_file} to {dest_subfolder}")
    else:
        print(f"No .cpp file found in {cpp_file}")


In [3]:
final_result = """### final result
                - **assertion result**: **pass** """
def SC_evaluate_assertion_with_api(SC_code, assertion):
    try:
        prompt = (
            "You are an expert in C/C++ programming and code analysis. Your task is to analyze whether the provided assertion passes or fails based on the given C/C++ code for a function.\n\n"
            "### C/C++ Code\n"
            "```\n"
            f"{SC_code}\n"
            "```\n\n"
            "### Assertion Statement\n"
            f"```\n{assertion}\n```\n\n"
            "Please write down your thinking process, and list the pass/fail result of each assertion at the end."
            "NOTE: Make sure the format the pass/fail result of each assertion at the end follow the example:"
            f"\n{final_result}"
        )
        # Call the OpenAI API with the prompt
        response =client.chat.completions.create(
            model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
            messages=[
                {"role": "user", "content": prompt},
            ],
            temperature=0.2,
        )

        # Extract the result
        result = response.choices[0].message.content.strip().lower()
        return result
    
    except Exception as e:
        if 'context_length_exceeded' in str(e):
            return "fail to read"
        else:
            return "error"

In [4]:
def extract_final_result(full_text):
    """
    Extracts the final result ("pass" or "fail") and comments from the analysis.
    """
    try:
        # Extract the final result (pass or fail) from the text
        final_result = "unknown"
        comment = full_text
        
        # Extract the final result line
        if "**assertion result**: **pass**" in full_text.lower():
            final_result = "pass"
        elif "**assertion result**: **fail**" in full_text.lower():
            final_result = "fail"

        return final_result, comment
    except Exception as e:
        return "error", str(e)

In [5]:
def process_folder(folder, relative_folder):
    SC_file = None
    csv_file = None
    
    # Identify the .bc and .csv files
    for file in os.listdir(folder):
        if file.endswith('.cpp'):
            SC_file = os.path.join(folder, file)
        elif file.endswith('.csv'):
            csv_file = os.path.join(folder, file)
    
    if not SC_file or not csv_file:
        print(f"Missing files in {folder}")
        return
    
    # Read the source code from the .bc file (assuming .cpp files are in LLVM bitcode text format)
    with open(SC_file, 'r') as file:
        SC_code = file.read()
    
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Prepare a list to store results
    results = []
    
    # Analyze each assertion statement
    for index, row in df.iterrows():
        assertion = row['assert_statement']
        
        # Evaluate the assertion using the OpenAI API
        full_analysis = SC_evaluate_assertion_with_api(SC_code, assertion)
        
        # Extract the final result and comment
        final_result, comment = extract_final_result(full_analysis)
        
        # Append to results list
        results.append([assertion, final_result, comment])
    
    # Create a DataFrame from the results
    result_df = pd.DataFrame(results, columns=['assert_statement', 'final_result', 'comment'])
    
    # Prepare the output folder
    output_folder = os.path.join(output_base_dir, relative_folder)
    os.makedirs(output_folder, exist_ok=True)
    
    # Define the output CSV file path
    output_file = os.path.join(output_folder, 'assertion_results.csv')
    
    # Save the results to a CSV file
    result_df.to_csv(output_file, index=False)

    print(f"Results saved to {output_file}")


In [8]:

# Example of how to call the function
# Define the base directories
input_base_dir = '/Users/jianfengzhu/Desktop/Researvh Work/02024AugLLMs_in_IRs/LLMs-in-IR-SEP7/processed_data/Human_x_164_O3'
output_base_dir = '/Users/jianfengzhu/Desktop/Researvh Work/02024AugLLMs_in_IRs/LLMs-in-IR-SEP7/processed_data/ScriptTest/DeepSeekFolder/DeepSeek_Execute_Result/Run3_DeepSeek_Execute'

# Ensure the output base directory exists
os.makedirs(output_base_dir, exist_ok=True)
# Assume each folder under the input_base_dir needs to be processed
for folder_name in os.listdir(input_base_dir):
    folder_path = os.path.join(input_base_dir, folder_name)
    if os.path.isdir(folder_path):
        process_folder(folder_path, folder_name)

Missing files in /Users/jianfengzhu/Desktop/Researvh Work/02024AugLLMs_in_IRs/LLMs-in-IR-SEP7/processed_data/Human_x_164_O3/png_files
Results saved to /Users/jianfengzhu/Desktop/Researvh Work/02024AugLLMs_in_IRs/LLMs-in-IR-SEP7/processed_data/ScriptTest/DeepSeekFolder/DeepSeek_Execute_Result/Run3_DeepSeek_Execute/CPP_47/assertion_results.csv
Results saved to /Users/jianfengzhu/Desktop/Researvh Work/02024AugLLMs_in_IRs/LLMs-in-IR-SEP7/processed_data/ScriptTest/DeepSeekFolder/DeepSeek_Execute_Result/Run3_DeepSeek_Execute/CPP_101/assertion_results.csv
Results saved to /Users/jianfengzhu/Desktop/Researvh Work/02024AugLLMs_in_IRs/LLMs-in-IR-SEP7/processed_data/ScriptTest/DeepSeekFolder/DeepSeek_Execute_Result/Run3_DeepSeek_Execute/CPP_78/assertion_results.csv
Results saved to /Users/jianfengzhu/Desktop/Researvh Work/02024AugLLMs_in_IRs/LLMs-in-IR-SEP7/processed_data/ScriptTest/DeepSeekFolder/DeepSeek_Execute_Result/Run3_DeepSeek_Execute/CPP_139/assertion_results.csv
Results saved to /Users/

In [33]:
def extract_final_result(full_text):
    """
    Extracts the final result ("pass" or "fail") and comments from the analysis.
    """
    try:
        # Extract the final result (pass or fail) from the text
        final_result = "unknown"
        comment = full_text
        
        # Extract the final result line
        if "**assertion result**: **pass**" in full_text.lower():
            final_result = "pass"
        elif "**assertion result**: **fail**" in full_text.lower():
            final_result = "fail"

        return final_result, comment
    except Exception as e:
        return "error", str(e)

In [34]:
def process_folder(folder, relative_folder):
    SC_file = None
    csv_file = None
    
    # Identify the .bc and .csv files
    for file in os.listdir(folder):
        if file.endswith('.cpp'):
            SC_file = os.path.join(folder, file)
        elif file.endswith('.csv'):
            csv_file = os.path.join(folder, file)
    
    if not SC_file or not csv_file:
        print(f"Missing files in {folder}")
        return
    
    # Read the source code from the .bc file (assuming .cpp files are in LLVM bitcode text format)
    with open(SC_file, 'r') as file:
        SC_code = file.read()
    
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Prepare a list to store results
    results = []
    
    # Analyze each assertion statement
    for index, row in df.iterrows():
        assertion = row['assert_statement']
        
        # Evaluate the assertion using the OpenAI API
        full_analysis = SC_evaluate_assertion_with_Tapi(SC_code, assertion)
        
        # Extract the final result and comment
        final_result, comment = extract_final_result(full_analysis)
        
        # Append to results list
        results.append([assertion, final_result, comment])
    
    # Create a DataFrame from the results
    result_df = pd.DataFrame(results, columns=['assert_statement', 'final_result', 'comment'])
    
    # Prepare the output folder
    output_folder = os.path.join(output_base_dir, relative_folder)
    os.makedirs(output_folder, exist_ok=True)
    
    # Define the output CSV file path
    output_file = os.path.join(output_folder, 'assertion_results.csv')
    
    # Save the results to a CSV file
    result_df.to_csv(output_file, index=False)

    print(f"Results saved to {output_file}")
